In [1]:
import keras
from keras.layers.core import Dense, Flatten
from keras.preprocessing.image import ImageDataGenerator
from keras.applications.vgg16 import VGG16
from keras.models import Model
from keras.applications.vgg16 import preprocess_input
from keras.preprocessing.image import load_img,img_to_array
from keras.models import model_from_json
import numpy as np
import os

Using TensorFlow backend.


In [2]:
train_path='/train'
test_path= '/test'

In [3]:
train_batches = ImageDataGenerator().flow_from_directory(train_path, target_size=(224,224), 
                                                         classes=['jablka','rajcata'], batch_size=10)
test_batches = ImageDataGenerator().flow_from_directory(test_path, target_size=(224,224), 
                                                        classes=['jablka','rajcata'],batch_size=4)

Found 40 images belonging to 2 classes.
Found 20 images belonging to 2 classes.


In [4]:
vgg16_model = VGG16(include_top=False, input_shape=(224, 224, 3))

Instructions for updating:
Colocations handled automatically by placer.


In [5]:
flat1 = Flatten()(vgg16_model.output)
output = Dense(2, activation='softmax')(flat1)

In [6]:
clf = Model(inputs=vgg16_model.inputs, outputs=output)

In [7]:
for layer in clf.layers:
    layer.trainable = False
for layer in clf.layers[-2:]:
    layer.trainable = True

In [8]:
clf.compile(optimizer = 'adam', loss='categorical_crossentropy', metrics = ['accuracy'] )

In [9]:
clf.fit_generator(train_batches, steps_per_epoch=4, validation_data=test_batches, validation_steps=10, epochs=3)

Instructions for updating:
Use tf.cast instead.
Epoch 1/3
4/4 [==============================] - 30s 8s/step - loss: 2.5955 - acc: 0.8000 - val_loss: 1.8033 - val_acc: 0.8500
Epoch 2/3
4/4 [==============================] - 31s 8s/step - loss: 0.4327 - acc: 0.9500 - val_loss: 1.6127 - val_acc: 0.9000
Epoch 3/3
4/4 [==============================] - 32s 8s/step - loss: 1.2468 - acc: 0.8750 - val_loss: 2.9012e-05 - val_acc: 1.0000


In [10]:
model_json = clf.to_json()

In [11]:
with open("clf.json", "w") as json_file:
    json_file.write(model_json)

In [12]:
clf.save_weights("model.h5")